# Data Preparation

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

# %cd '/content/drive/My Drive/ML1_2024_2025/02'

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

In [8]:
houses = pd.read_csv('Data/houses.csv')
houses.head()

,Order,PID,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,1,526301100,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,...,No_Fence,NaN,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,2,526350040,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,...,Minimum_Privacy,NaN,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,3,526351010,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,...,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,4,526353030,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,...,No_Fence,NaN,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,5,527105010,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,...,Minimum_Privacy,NaN,0,3,2010,WD,Normal,189900,-93.638933,42.060899


In [10]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 83 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order               2930 non-null   int64  
 1   PID                 2930 non-null   int64  
 2   MS_SubClass         2930 non-null   object 
 3   MS_Zoning           2930 non-null   object 
 4   Lot_Frontage        2930 non-null   int64  
 5   Lot_Area            2930 non-null   int64  
 6   Street              2930 non-null   object 
 7   Alley               2930 non-null   object 
 8   Lot_Shape           2930 non-null   object 
 9   Land_Contour        2930 non-null   object 
 10  Utilities           2930 non-null   object 
 11  Lot_Config          2930 non-null   object 
 12  Land_Slope          2930 non-null   object 
 13  Neighborhood        2930 non-null   object 
 14  Condition_1         2930 non-null   object 
 15  Condition_2         2930 non-null   object 
 16  Bldg_T

### Missing Values

In [20]:
columns_with_missings = houses.columns[houses.isnull().any()]
houses[columns_with_missings].isnull().sum()

Mas_Vnr_Type    1775
Electrical         1
Misc_Feature    2824
dtype: int64

### Frequency Tables for these columns

In [43]:
print('--------------------------------------------------------')
print('Frequency counts for Mas_Vnr_Type')
print(houses['Mas_Vnr_Type'].value_counts(dropna=False))
print('--------------------------------------------------------')
print('Frequency counts for Electrical')
print(houses['Electrical'].value_counts(dropna=False))
print('--------------------------------------------------------')
print('Frequency counts for Misc_Feature')
print(houses['Misc_Feature'].value_counts(dropna=False))
print('--------------------------------------------------------')

--------------------------------------------------------
Frequency counts for Mas_Vnr_Type
Mas_Vnr_Type
NaN        1775
BrkFace     880
Stone       249
BrkCmn       25
CBlock        1
Name: count, dtype: int64
--------------------------------------------------------
Frequency counts for Electrical
Electrical
SBrkr    2682
FuseA     188
FuseF      50
FuseP       8
NaN         1
Mix         1
Name: count, dtype: int64
--------------------------------------------------------
Frequency counts for Misc_Feature
Misc_Feature
NaN     2824
Shed      95
Gar2       5
Othr       4
Elev       1
TenC       1
Name: count, dtype: int64
--------------------------------------------------------


In [47]:
# according to data description the missing value in Mas_Vnr_Type and in Misc_Feature should be the value "None"
# in case of Electrical column we can replace the missing value with the most frequent level "SBrkr"

# All these can be done BEFORE the train/test aplit as they do not depend on the specific data distribution

houses['Mas_Vnr_Type'] = houses['Mas_Vnr_Type'].fillna('None')
houses['Misc_Feature'] = houses['Misc_Feature'].fillna('None')

houses['Electrical'] = houses['Electrical'].fillna('SBrkr')

In [49]:
# frequency table again to check the results of imputation
print('--------------------------------------------------------')
print('Frequency counts for Mas_Vnr_Type')
print(houses['Mas_Vnr_Type'].value_counts(dropna=False))
print('--------------------------------------------------------')
print('Frequency counts for Electrical')
print(houses['Electrical'].value_counts(dropna=False))
print('--------------------------------------------------------')
print('Frequency counts for Misc_Feature')
print(houses['Misc_Feature'].value_counts(dropna=False))
print('--------------------------------------------------------')

--------------------------------------------------------
Frequency counts for Mas_Vnr_Type
Mas_Vnr_Type
None       1775
BrkFace     880
Stone       249
BrkCmn       25
CBlock        1
Name: count, dtype: int64
--------------------------------------------------------
Frequency counts for Electrical
Electrical
SBrkr    2683
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: count, dtype: int64
--------------------------------------------------------
Frequency counts for Misc_Feature
Misc_Feature
None    2824
Shed      95
Gar2       5
Othr       4
Elev       1
TenC       1
Name: count, dtype: int64
--------------------------------------------------------


In [51]:
# check for missing values after imputation
columns_with_missings = houses.columns[houses.isnull().any()]
houses[columns_with_missings].isnull().sum()
# Result: no more missings

Series([], dtype: float64)

### Distributions of nominal variables

Lets check the frequencies of individual levels of all the nominal variables.

Extremely small counts of single levels are problematic as they might be very rare cases which a model will not be able to learn how they influence the sale price.

Therefore we will replace all the levels which have no more than 25 observations with the label "other".


In [59]:
# lets check the frequencies of individual levels of all the nominal variables
nominal_variables = ['MS_SubClass', 'MS_Zoning', 'Street', 'Alley', 'Lot_Config',
                     'Neighborhood', 'Condition_1', 'Condition_2', 'Bldg_Type',
                     'House_Style', 'Roof_Style', 'Roof_Matl', 'Exterior_1st',
                     'Exterior_2nd', 'Mas_Vnr_Type', 'Foundation', 'Misc_Feature',
                     'Heating', 'Central_Air', 'Garage_Type', 'Misc_Feature',
                     'Sale_Type', 'Sale_Condition', 'Land_Contour']

# loop through each nominal variable
for var in nominal_variables:
    value_counts = houses[var].value_counts()
    print('\n')
    print(houses[var].value_counts(dropna=False))



MS_SubClass
One_Story_1946_and_Newer_All_Styles          1079
Two_Story_1946_and_Newer                      575
One_and_Half_Story_Finished_All_Ages          287
One_Story_PUD_1946_and_Newer                  192
One_Story_1945_and_Older                      139
Two_Story_PUD_1946_and_Newer                  129
Two_Story_1945_and_Older                      128
Split_or_Multilevel                           118
Duplex_All_Styles_and_Ages                    109
Two_Family_conversion_All_Styles_and_Ages      61
Split_Foyer                                    48
Two_and_Half_Story_All_Ages                    23
One_and_Half_Story_Unfinished_All_Ages         18
PUD_Multilevel_Split_Level_Foyer               17
One_Story_with_Finished_Attic_All_Ages          6
One_and_Half_Story_PUD_All_Ages                 1
Name: count, dtype: int64


MS_Zoning
Residential_Low_Density         2273
Residential_Medium_Density       462
Floating_Village_Residential     139
Residential_High_Density          27


In [71]:
# lets replace all the levels which have no more than 25 observations (ca. 1% of the sample size) with the label "other"

# IMPORTANT! Remember that for a NOMINAL variable we can combine any levels together,
# while for an ORIDAL variable (feature) ONLY the neighboring levels can be combined!

# based on the information about the dataset we can list all nominal variables

# loop through each nominal variable
for var in nominal_variables:
    value_counts = houses[var].value_counts()
    levels_to_replace = value_counts[value_counts <= 25].index
    houses[var] = houses[var].replace(levels_to_replace, 'Other')

In [74]:
# lets check the frequencies of nominal variables again
for var in nominal_variables:
    value_counts = houses[var].value_counts()
    print('\n')
    print(houses[var].value_counts(dropna=False))

# Result: looks like the problem with small counts has been solved



MS_SubClass
One_Story_1946_and_Newer_All_Styles          1079
Two_Story_1946_and_Newer                      575
One_and_Half_Story_Finished_All_Ages          287
One_Story_PUD_1946_and_Newer                  192
One_Story_1945_and_Older                      139
Two_Story_PUD_1946_and_Newer                  129
Two_Story_1945_and_Older                      128
Split_or_Multilevel                           118
Duplex_All_Styles_and_Ages                    109
Other                                          65
Two_Family_conversion_All_Styles_and_Ages      61
Split_Foyer                                    48
Name: count, dtype: int64


MS_Zoning
Residential_Low_Density         2273
Residential_Medium_Density       462
Floating_Village_Residential     139
Other                             29
Residential_High_Density          27
Name: count, dtype: int64


Street
Pave     2918
Other      12
Name: count, dtype: int64


Alley
No_Alley_Access    2732
Gravel              120
Paved             

### Near-Zero Variance
Near-zero variance (NZV) predictors are features in a dataset that have either only a single unique value or very few unique values relative to the total number of samples

What can be still problematic however is the **strong concentration of variable values in one level** (issue called: **near-zero variance**) - such variables have little predictive power and can be omitted

To identify variables that have a small variance one can use a measure based on the ratio of the most frequent and second most frequent values (**frequency ratio**)/

For well diversified variables it will take a value close to 1, and a very high value for unbalanced data.

Another useful measure is the **percent of unique values**: `100 * (number of unique values) / (number of observations)`

It is close to zero for data that is not very diversified.

If the "frequency ratio" is greater than some threshold, and the "percent of unique values" is less than a certain limit value, we can assume that the variable has a variance close to 0 and can be potentially omitted.

Lets define a function which will check it with the threshold 95/5 for the **frequency ratio** and 10 for the **percent of unique values**.